In [1]:
import pandas as pd
import numpy as np
import wfdb
import ast
import time
from itertools import chain
from collections import Counter
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import acf
import os
import tqdm
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, auc, f1_score, roc_curve, RocCurveDisplay
from preprocessing import preprocess_data

In [2]:
x_train, x_test, y_train, y_test = preprocess_data(path='plt')

Loading data


100%|██████████| 21799/21799 [03:34<00:00, 101.86it/s]


Dropping other
Filtering outliers


100%|██████████| 14538/14538 [00:16<00:00, 900.77it/s] 


Applying moving average


100%|██████████| 13792/13792 [01:46<00:00, 129.76it/s]


Balancing data
Train-test split


### Обучение модели

Обучение сети

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
class ConvNet(nn.Module):
    def __init__(
        self,
        fc_layers_size,
        conv_layers
    ) -> None:
        super(ConvNet, self).__init__()
        layers = []
        in_channels = 12
        conv_out_size = 1000
        for kernel_size, out_channels, stride in conv_layers:
            layers += [
                nn.Conv1d(in_channels, out_channels, kernel_size, stride),
                nn.ReLU(),
                nn.MaxPool1d(2, 2)
            ]
            in_channels = out_channels
            # for Conv1d
            conv_out_size = self.conv_eval(conv_out_size, kernel_size, stride, 1)
            # for MaxPool1d
            conv_out_size = self.conv_eval(conv_out_size, 2, 2, 1)
        layers.append(nn.Flatten())
        features_in = conv_out_size * out_channels
        for fc_out in fc_layers_size[:-1]:
            layers += [
                nn.Linear(features_in, fc_out),
                nn.ReLU()
            ]
            features_in = fc_out
        layers += [
            nn.Linear(features_in, fc_layers_size[-1]),
            nn.Sigmoid()
        ]
        self.seq = nn.Sequential(*layers)
    
    @classmethod
    def conv_eval(self, len_in, kernel_size, stride, dilation):
        return int((len_in - 1 - dilation * (kernel_size - 1)) / stride + 1)
    
    def forward(self, x):
        return self.seq(x)
    
    def train_net(
        self,
        criterion,
        optimizer,
        objects,
        labels,
        epochs,
        batch_size
    ):
        objects = torch.Tensor(objects)
        labels = torch.Tensor(np.array([labels]).T)
        dataset = TensorDataset(objects, labels)
        dataloader = DataLoader(dataset, batch_size, shuffle=True)
        test_accuracy_list = [0]

        for epoch in range(epochs):
            loss_value = 0.0
            for i, batch in enumerate(dataloader):
                inputs, labels = batch[0].to(device, non_blocking=True), batch[1].to(device, non_blocking=True)
                optimizer.zero_grad()
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                loss_value += loss.item()
            train_accuracy = accuracy_score(
                y_train,#.to(device, non_blocking=True), 
                [(1 if pred[0] > 0.5 else 0) for pred in self.test(torch.Tensor(x_train).to(device, non_blocking=True))]
            )
            test_accuracy = accuracy_score(
                y_test,#.to(device, non_blocking=True), 
                [(1 if pred[0] > 0.5 else 0) for pred in self.test(torch.Tensor(x_test).to(device, non_blocking=True))]
            )
            test_accuracy_list.append(test_accuracy)
            print(f'Epoch {epoch:5} loss = {loss_value:.4f} Train: {train_accuracy:.4f} Test {test_accuracy:.4f}')
            #if epoch > 5 and np.average(test_accuracy_list[-4:]) < test_accuracy_list[-5]:
            #    print('Test accuracy decreasing, break')
            #    break
    
    def test(self, samples):
        with torch.no_grad():
            return self.forward(torch.Tensor(samples))

In [5]:
generated = ConvNet(
    fc_layers_size=[500, 1],
    conv_layers=[
        (7, 48, 2),
        (5, 144, 2),
        (5, 288, 2),
    ]
)
generated
generated.to(device)
generated.train_net(
    nn.BCELoss(), 
    optim.Adam(generated.parameters(), lr=0.0001),
    x_train, 
    y_train, 
    30, 
    128
)

Epoch     0 loss = 35.1395 Train: 0.7318 Test 0.7309
Epoch     1 loss = 24.8638 Train: 0.8332 Test 0.8288
Epoch     2 loss = 20.2137 Train: 0.8507 Test 0.8388
Epoch     3 loss = 17.9933 Train: 0.8697 Test 0.8603
Epoch     4 loss = 17.1632 Train: 0.8808 Test 0.8679
Epoch     5 loss = 16.0666 Train: 0.8854 Test 0.8726


KeyboardInterrupt: 